# VacationPy

### Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,60.82,84,8,5.01,NZ,1699935855
1,1,big bend,-26.8167,31.9333,62.69,94,100,2.30,SZ,1699935856
2,2,longyearbyen,78.2186,15.6401,12.04,73,40,5.75,SJ,1699935856
3,3,talnakh,69.4865,88.3972,-6.86,96,40,6.13,RU,1699935856
4,4,chonchi,-42.6128,-73.8092,38.41,92,79,3.33,CL,1699935858


### Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "Humidity",
    cmap = "viridis"
)

# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                   & (city_data_df["Max Temp"] > 21)
                                   & (city_data_df["Wind Speed"] < 4.5)
                                   & (city_data_df["Cloudiness"] > 90)]
                                   
 
# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,fort mcmurray,56.7268,-111.3810,26.19,93,100,0.00,CA,1699935859
72,72,college,64.8569,-147.8028,26.55,91,100,0.00,US,1699935653
126,126,badger,64.8000,-147.5333,26.47,92,100,0.00,US,1699935902
170,170,kurumkan,54.3000,110.3000,22.12,86,100,1.01,RU,1699935920


### Step 3: Create a new DataFrame called hotel_df.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,fort mcmurray,CA,56.7268,-111.3810,93,
72,college,US,64.8569,-147.8028,91,
126,badger,US,64.8000,-147.5333,92,
170,kurumkan,RU,54.3000,110.3000,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
          }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort mcmurray - nearest hotel: No hotel found
college - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
kurumkan - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
8,fort mcmurray,CA,56.7268,-111.3810,93,No hotel found
72,college,US,64.8569,-147.8028,91,No hotel found
126,badger,US,64.8000,-147.5333,92,No hotel found
170,kurumkan,RU,54.3000,110.3000,86,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", 
                                   geo = True, 
                                   tiles="OSM",
                                   color = "City",
                                   hover_cols = ["Hotel Name","Country"])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)